# sandbox to test stuff

In [14]:
import pandas as pd
from src.user_lib.read_file import *

In [15]:
pathtofile = 'Rechnungen_Stadtmobil.xlsx'
sheetname = 'Rechnungen_Stadtmobil'
df = readDataFromXls(pathtofile,sheetname)
df.head(5)

,Kunde/Kostenstelle,Name,Fahrzeug/Typ,Von,Bis,km,km-/Kraftstoffpreis,Zeitpreis,Preis,Mwst,Bemerkung,Erklärung,Konto,Kostenstelle,Fahrzeugtyp,Kennzeichen,Station
0,13523,Eger Jana,NaN,2024-02-01,2024-12-31 23:59:59,NaN,NaN,NaN,27.5,19,NaN,NaN,Monatsbeitrag,NaN,NaN,NaN,NaN
1,13523,Eger Jana,NaN,2024-02-01,2025-01-31 23:59:59,NaN,NaN,NaN,25.0,19,NaN,NaN,Sicherheitspaket,NaN,NaN,NaN,NaN
2,13523,Eger Jana,NaN,2024-03-01,2025-02-28 23:59:59,NaN,NaN,NaN,12.0,0,NaN,NaN,Mitgliedsbeitrag Ökostadt,NaN,NaN,NaN,NaN
3,13523,Eger Jana,NaN,2024-02-26,2024-02-26 00:00:00,NaN,NaN,NaN,35.0,19,NaN,NaN,Tarifstart,NaN,NaN,NaN,NaN
4,13520,Geyer Wolfgang,NaN,2024-02-01,2024-12-31 23:59:59,NaN,NaN,NaN,55.0,19,NaN,NaN,Monatsbeitrag,NaN,NaN,NaN,NaN


In [16]:
# Convert the 'Von' and 'Bis' columns to datetime
df['Von'] = pd.to_datetime(df['Von'])
df['Bis'] = pd.to_datetime(df['Bis'])

# Calculate the duration (Dauer) and add it as a new column
df['Dauer'] = df['Bis'] - df['Von']
# Calculate the duration in hours
df['Dauer_hours'] = df['Dauer'].dt.total_seconds() / 3600

# Ensure the relevant columns are numeric
df['km'] = pd.to_numeric(df['km'], errors='coerce')
df['km-/Kraftstoffpreis'] = pd.to_numeric(df['km-/Kraftstoffpreis'], errors='coerce')
df['Preis'] = pd.to_numeric(df['Preis'], errors='coerce')
df['Zeitpreis'] = pd.to_numeric(df['Zeitpreis'], errors='coerce')


# Modify the 'Konto' column based on the condition in 'Erklärung'
df['Konto'] = df.apply(lambda row: f"{row['Konto']}_Storno" if 'Storno' in str(row['Erklärung']) else row['Konto'], axis=1)

# Sort the DataFrame by 'Von' in ascending order
df = df.sort_values('Von').reset_index(drop=True)

# Create a new column 'Buchungsnr' by numbering the unique 'Von' entries
df['Buchungsnr'] = df.groupby('Von').ngroup() + 1

# If you want to reset the index after sorting, you can add this line:
df.reset_index(drop=True, inplace=True)
df.sample(3)

,Kunde/Kostenstelle,Name,Fahrzeug/Typ,Von,Bis,km,km-/Kraftstoffpreis,Zeitpreis,Preis,Mwst,Bemerkung,Erklärung,Konto,Kostenstelle,Fahrzeugtyp,Kennzeichen,Station,Dauer,Dauer_hours,Buchungsnr
21,13520,Geyer Wolfgang,"Toyota Corolla Hybrid Kombi, MA-CS 4552, Mannh...",2024-03-16 16:30:00,2024-03-16 18:00:00,16.0,NaN,1.00,1.00,19,NaN,pauschaler Betrag pro Buchung,Buchungspauschale,NaN,Toyota Corolla Hybrid Kombi,MA-CS 4552,"Mannheim, Johann-Weiß-Str. 9",0 days 01:30:00,1.5,7
15,13520,Geyer Wolfgang,"Toyota Corolla Hybrid Kombi, MA-CS 4552, Mannh...",2024-03-04 17:30:00,2024-03-04 20:00:00,87.0,NaN,7.13,7.13,19,NaN,"2,5 Stunden zu je 2,85 EUR.",Zeitkosten,NaN,Toyota Corolla Hybrid Kombi,MA-CS 4552,"Mannheim, Johann-Weiß-Str. 9",0 days 02:30:00,2.5,6
35,13520,Geyer Wolfgang,"Toyota Corolla Hybrid Kombi, MA-CS 4552, Mannh...",2024-06-24 15:30:00,2024-06-24 20:30:00,29.0,NaN,1.00,1.00,19,NaN,pauschaler Betrag pro Buchung,Buchungspauschale,NaN,Toyota Corolla Hybrid Kombi,MA-CS 4552,"Mannheim, Johann-Weiß-Str. 9",0 days 05:00:00,5.0,11


In [17]:
# Sum up all prices and kilometers
total_price = df['Preis'].sum()

df_ = df[df['km-/Kraftstoffpreis'].isna()==False]
total_km = df_['km'].sum()

# Calculate the average price per km
average_price_per_km = total_price / total_km if total_km != 0 else None

# Print the results
print(f"Total Price: {total_price}")
print(f"Total km: {total_km}")
print(f"Average Price per km: {average_price_per_km}")


Total Price: 771.5999999999999
Total km: 1086.0
Average Price per km: 0.7104972375690607


In [18]:
import plotly.express as px

# Assuming df_fahrten is your DataFrame

# Group by 'Konto' and sum the 'Preis' column
konto_sums = df.groupby('Konto')['Preis'].sum().reset_index()


# Format the title with the calculated values
title = (f'Stadtmobil Statistics<br>'
         f'Total Price: €{total_price:.2f}<br>'
         f'Total km: {total_km:.0f}<br>'
         f'avg Price per km: €{average_price_per_km:.2f}')

# Create a pie chart with labels showing the sum for each category
fig = px.pie(konto_sums, values='Preis', names='Konto',
             title=title,
             labels={'Preis': 'Total Price', 'Konto': 'Account'},
             hole=0.3)  # Optional: Adds a hole in the center for a donut chart look

# Update the pie chart to show the sum in the labels
fig.update_traces(textinfo='label+value+percent', 
                  textfont_size=14, 
                  textposition='outside')

## Add annotations with total price, total kilometers, and average price per km
#fig.update_layout(
#    annotations=[
#        dict(
#            x=-0.,  # X position of the annotation
#            y=0.2,  # Y position of the annotation (below the chart)
#            text=f"Total Price: €{total_price:.2f}<br>Total km: {total_km:.2f}<br>Average Price per km: €{average_price_per_km:.2f}",
#            showarrow=True,
#            font=dict(size=14, color="black"),
#            align="center",
#            xanchor='center',
#            yanchor='middle',
#            borderpad=4,
#            bgcolor="rgba(255, 255, 255, 0.8)",
#            bordercolor="black"
#        )
#    ]
#)

# Show the figure
fig.show()


In [19]:
df_fahrten = df[df['km'].isna()==False]
df_fahrten.sample()

,Kunde/Kostenstelle,Name,Fahrzeug/Typ,Von,Bis,km,km-/Kraftstoffpreis,Zeitpreis,Preis,Mwst,Bemerkung,Erklärung,Konto,Kostenstelle,Fahrzeugtyp,Kennzeichen,Station,Dauer,Dauer_hours,Buchungsnr
49,13520,Geyer Wolfgang,"Renault ZOE Elektro, MA-CS 301E, Mannheim, Fri...",2024-07-07 11:00:00,2024-07-07 14:00:00,41.0,NaN,7.05,7.05,19,NaN,"3 Stunden zu je 2,35 EUR.",Zeitkosten,NaN,Renault ZOE Elektro,MA-CS 301E,"Mannheim, Fritz-Salm-Straße 12 (Turley)",0 days 03:00:00,3.0,15


In [20]:

# Sort the DataFrame by 'Von' in ascending order
df_fahrten = df_fahrten.sort_values('Von').reset_index(drop=True)

# Create a new column 'Buchungsnr' by numbering the unique 'Von' entries
df_fahrten['Buchungsnr'] = df_fahrten.groupby('Von').ngroup() + 1

# If you want to reset the index after sorting, you can add this line:
df_fahrten.reset_index(drop=True, inplace=True)



In [21]:
import plotly.express as px

# Assuming df is your DataFrame and the columns are already processed

# Create the bar plot
fig = px.bar(df_fahrten, x='Buchungsnr', y='Preis', color='Konto', 
             title='Price over Time by Konto',
             labels={'Von': 'Start Date', 'Preis': 'Price', 'Konto': 'Account'},
             #barmode='group')
             barmode='relative')  # 'relative' makes the bars stacked)

# Show the figure
fig.show()

In [23]:
import pandas as pd
import plotly.express as px

# Sample DataFrame (replace this with your actual DataFrame)
# df_fahrten = pd.DataFrame({...})

# Ensure the 'Von' column is in datetime format
df['Von'] = pd.to_datetime(df['Von'], errors='coerce')

# Extract the month and year from the 'Von' column and convert to string format
df['Month_Year'] = df['Von'].dt.to_period('M').astype(str)

# Aggregate prices by 'Month_Year' and 'Name'
monthly_prices = df.groupby(['Month_Year', 'Name'])['Preis'].sum().reset_index()

# Pivot the DataFrame to have separate columns for each 'Name'
pivot_df = monthly_prices.pivot_table(index='Month_Year', columns='Name', values='Preis', fill_value=0).reset_index()

# Create a stacked column chart
fig = px.bar(pivot_df, x='Month_Year', y=pivot_df.columns[1:],  # Exclude 'Month_Year' column from y-axis
             title='Price per Month for each Name',
             labels={'value': 'Total Price', 'Month_Year': 'Month/Year'},
             color_discrete_sequence=px.colors.qualitative.Plotly)

# Show the figure
fig.show()



In [24]:
import pandas as pd
import plotly.express as px

# Sample DataFrame (replace this with your actual DataFrame)
# df_fahrten = pd.DataFrame({...})

# Ensure the 'Von' column is in datetime format
df['Von'] = pd.to_datetime(df['Von'], errors='coerce')

# Extract the month and year from the 'Von' column and convert to string format
df['Month_Year'] = df['Von'].dt.to_period('M').astype(str)

# Aggregate prices by 'Month_Year', 'Name', and 'Konto'
monthly_prices = df.groupby(['Month_Year', 'Name', 'Konto'])['Preis'].sum().reset_index()

# Create a stacked column chart with 'Name' as separate columns and 'Konto' as color
fig = px.bar(monthly_prices, x='Month_Year', y='Preis', color='Konto',
             title='Price per Month for each Name',
             labels={'Month_Year': 'Month/Year', 'Preis': 'Total Price', 'Konto': 'Account'},
             facet_col='Name',  # Create separate columns for each 'Name'
             color_discrete_sequence=px.colors.qualitative.Plotly)

# Show the figure
fig.show()
